In [28]:
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np

In [29]:
def cnn_model_fn(features, labels, mode):
    #input layer
    input_layer= tf.reshape(features["x"], [-1,28,28,1])
    
    #convolutional layer #1
    
    conv1= tf.layers.conv2d(inputs= input_layer,
                            filers=32,
                            kernel_size=[5,5],
                           padding="same",
                           activation=tf.nn.relu)
    
    #pooling layer #1
    pool1= tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2]. stride=2)
    
    #convolutional layer #2 and pooling layer #2
    
    conv2= tf.layers.conv2d(inputs=poool1,
                            filters= 64,
                            kernel_size=[5,5],
                            padding="same",
                            activation= tf.nn.relu
                           )
    pool2= tf.layers.max_pooling2d(inputs= conv2, pool_size=[2,2], strides=2)
    
    # Dense layer
    pool2_flat= tf.reshape(pool2, [-1, 7*7*64])
    dense= tf.layers.dense(iputs=pool2_flat, units=1024, activation= tf.nn.relu)
    dropout= tf.layers.dropout(inputs_dense, rate= .4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # logits layer
    logits= tf.layers.dense(inputs= dropout, units=10)
    perdictions= {
        # generate predictions ( for predict and eval mode)
        "classes": tf.argmax(inputs=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimateSpec(mode=mode, preductions= predictions)
    
    # calcualtion loss (for both train and eval modes)
    loss= tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits )
    
    # configure the training op( for train mode)

    if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

SyntaxError: invalid syntax (<ipython-input-29-78ae3bafc3d1>, line 14)

In [60]:
def dense_block(features):

    #convolutional layer 2
    conv1= features
    conv2= tf.layers.conv2d(inputs= conv1,
                            filters=12,
                            kernel_size=[3,3],
                           padding="same",
                           activation=tf.nn.relu)
    
    #concatenation   
    concat1= tf.concat([conv1,conv2],-1)
    
    # conv layer 3
    
    conv3a= tf.layers.conv2d(inputs= conv2, filters=12*4, kernel_size=[1,1],
                         padding="same", activation=tf.nn.relu)
    
    conv3= tf.layers.conv2d(inputs= conv3a, filters=12, kernel_size=[3,3],
                         padding="same", activation=tf.nn.relu)

    #concatenation
    concat2= tf.concat([concat1, conv3], -1) 
    return concat2
    

    
def transition_layer(features):
        # conv layer 1
    
    conv1= tf.layers.conv2d(inputs= features,
                            filters=int(features.shape[-1]),
                            kernel_size=[5,5],
                           padding="same",
                           activation=tf.nn.relu)
    
    pool1= tf.layers.max_pooling2d(inputs= conv1, pool_size=[2,2], strides=2)
    
    return pool1
    
    

In [69]:
def denseNet(x, numblocks):
    x1 = tf.convert_to_tensor(x, np.float32)
    # conv layer 1
    output= tf.layers.conv2d(inputs= x1,
                                filters=24,
                                kernel_size=[1,1],
                               padding="same",
                               activation=tf.nn.relu)

    for i in range(numblocks-1):
        output= dense_block(output)
        output= transition_layer(output)

    dense= dense_block(output)
    pool1= tf.layers.max_pooling2d(inputs= dense,pool_size=[2,2], strides=2)

    # fully connected layers
    pool1_flat= tf.reshape(pool1, [-1,int(pool1.shape[-1])*int(pool1.shape[-2])*int(pool1.shape[-3])])
    dense= tf.layers.dense(inputs= pool1_flat, units=1024, activation=tf.nn.relu)
    dropout= tf.layers.dropout(inputs=dense, rate= .4)
    logits= tf.layers.dense(inputs= dropout, units=10)
    
    return logits
    

In [70]:
#x= np.random.rand(200,28,28,3)

# plance holders
x = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28, 3])
labels = tf.placeholder(dtype = tf.int32, shape = [None])
logits= denseNet(x,2)
perdiction= tf.argmax(logits, name="softmax tensor")

# calcualtion loss (for both train and eval modes)
loss= tf.losses.sparse_softmax_cross_entropy(labels=labels, logits= logits)

#optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())


In [71]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(epoch):
    perdiction, loss= sess.run([perdiction, loss], feed_dict={x:X, labels: y})

<tf.Tensor 'dense_4/BiasAdd:0' shape=(200, 10) dtype=float32>